In [1]:
import pickle
from os.path import join
import pandas as pd
from copy import deepcopy
import re
from glob import glob
import os
import torch

In [2]:
data_dir = "/nfs/team/nlp/users/rgupta/NMT/code/GNN-Semantic-Similarity/data"

In [3]:
bpe_vs = ['500', '10000', '50000']
word_vs = ['words-lower', 'words-mix']
word_vocab_sizes = [5000, 10000, 25000]
models = ['bilstm', 'gcn_head']

In [4]:
def merge_dicts(d1, d2):
    res = deepcopy(d1)
    for k, v in d2.items():
        res[k] = v
    return res

In [5]:
def parse_file(f):
    if isinstance(f, str) and os.path.exists(f):
        with open(f) as f:
            lines = [l.strip() for l in f.readlines()]
    else:
        lines = [l.strip() for l in f.readlines()]
        
    res = {}
    buff = []
    covered = []
    keys = []
    for line in lines:
        line = line.split('|')[1]
        if "LR:" in line:
            if line not in covered:
                covered.append(line)
                key = line + ", AGGR: mean"
            else:
                key = line + ", AGGR: max"
            if keys:
                res[keys[-1]] = buff
            keys.append(key)
            buff = []
        elif "Epoch:" in line:
            buff.append(dict([param.strip().split(':') for param in line.split(',')]))
    
    res[key] = buff
    
    return res 

In [6]:
res = parse_file(join(data_dir, f"logs/gcn_head_vocab_10000_neg_sample_1_grid_search.log.log"))

## Bilstm

In [7]:
bilstm_pd_list = []

In [8]:
bilstm_dict = {}

for bpe_v in bpe_vs:
    with open(join(data_dir, f"grid_search/bilstm_vocab_{bpe_v}_neg_sample_1_grid_search.pkl"), 'rb') as f:
        bilstm_dict[bpe_v] = pickle.load(f)

for word_v in word_vs:
    for word_vocab_size in word_vocab_sizes:
        with open(join(data_dir, f"grid_search/bilstm_vocab_{word_v}_neg_sample_1_vocab_size_{word_vocab_size}_grid_search.pkl"), 'rb') as f:
            bilstm_dict[f'{word_v}|{word_vocab_size}'] = pickle.load(f)

In [9]:
for v, res_d in bilstm_dict.items():
    if v.startswith('word'):
        vocab, size = v.split("|")
        d = {'head_vocab': vocab, "word_vocab_size": size, "side_vocabs": "NA", "AGGR": "NA"}
    else:
        d = {'head_vocab': v, "word_vocab_size": -1, "side_vocabs": "NA", "AGGR": "NA"}
    for key, losses in res_d.items():
        d.update(dict([param.strip().split(':') for param in key.split(',')]))
        for epoch, loss_d in enumerate(losses):
            if not isinstance(loss_d, dict):
                loss_d = {"Accuracy": loss_d, "Train Loss": -1, "Valid Loss": -1}
            new_d = merge_dicts({'epoch': epoch, 'model': 'bilstm'}, loss_d)
            bilstm_pd_list.append(merge_dicts(d, new_d))

In [10]:
accs = sorted([float(rec['Accuracy'].numpy()) for rec in bilstm_pd_list], reverse=True)
len(accs), accs[:10]

(7039,
 [0.9615293741226196,
  0.9608191251754761,
  0.9606415629386902,
  0.9604048132896423,
  0.9603456258773804,
  0.9601088762283325,
  0.9599905014038086,
  0.9598721861839294,
  0.9597538113594055,
  0.9596354365348816])

## GCN

### Single head chain

In [11]:
gcn_pd_list = []

In [12]:
gcn_dict = {}

for bpe_v in bpe_vs:
    with open(join(data_dir, f"logs/gcn_head_vocab_{bpe_v}_neg_sample_1_grid_search.log.log"), 'r') as f:
        gcn_dict[bpe_v] = parse_file(f)

for word_v in word_vs:
    for word_vocab_size in word_vocab_sizes:
        with open(join(data_dir, f"logs/gcn_head_vocab_{word_v}_neg_sample_1_vocab_size_{word_vocab_size}_grid_search.log.log"), 'r') as f:
            gcn_dict[f'{word_v}|{word_vocab_size}'] = parse_file(f)

In [13]:
for v, res_d in gcn_dict.items():
    if v.startswith('word'):
        vocab, size = v.split("|")
        d = {'head_vocab': vocab, "word_vocab_size": size, "side_vocabs": "NA"}
    else:
        d = {'head_vocab': v, "word_vocab_size": -1, "side_vocabs": "NA"}
    for key, losses in res_d.items():
        d.update(dict([param.strip().split(':') for param in key.split(',')]))
        for epoch, loss_d in enumerate(losses):
            new_d = merge_dicts({'epoch': epoch, 'model': 'gcn'}, loss_d)
            gcn_pd_list.append(merge_dicts(d, new_d))

In [14]:
gcn_dict.keys()

dict_keys(['500', '10000', '50000', 'words-lower|5000', 'words-lower|10000', 'words-lower|25000', 'words-mix|5000', 'words-mix|10000', 'words-mix|25000'])

In [15]:
accs = sorted([float(rec['Accuracy'].strip()) for rec in gcn_pd_list], reverse=True)
len(accs), accs[:10]

(2880,
 [0.961,
  0.9601,
  0.9596,
  0.9595,
  0.9592,
  0.9592,
  0.9587,
  0.9585,
  0.9584,
  0.958])

### 1 head and 1 BPE Chain

In [16]:
gcn_dict = {}

for word_v in word_vs:
    for bpe_v in bpe_vs:
        for word_vocab_size in word_vocab_sizes:
            with open(join(data_dir, f"logs/gcn_head_vocab_{word_v}_bpe_vocab_{bpe_v}_neg_sample_1_vocab_size_{word_vocab_size}_grid_search.log.log"), 'r') as f:
                gcn_dict[f'{word_v}|{bpe_v}|{word_vocab_size}'] = parse_file(f)

In [17]:
for v, res_d in gcn_dict.items():
    word_v, bpe_v, size = v.split("|")
    d = {'head_vocab': vocab, "word_vocab_size": size, "side_vocabs": bpe_v}
    for key, losses in res_d.items():
        d.update(dict([param.strip().split(':') for param in key.split(',')]))
        for epoch, loss_d in enumerate(losses):
            new_d = merge_dicts({'epoch': epoch, 'model': 'gcn'}, loss_d)
            gcn_pd_list.append(merge_dicts(d, new_d))

In [18]:
accs = sorted([float(rec['Accuracy'].strip()) for rec in gcn_pd_list], reverse=True)
len(accs), accs[:10]

(8640,
 [0.961,
  0.9601,
  0.9596,
  0.9595,
  0.9592,
  0.9592,
  0.9587,
  0.9585,
  0.9584,
  0.958])

### 1 head and multiple BPE Chains

In [19]:
f_names = glob(join(data_dir, f"logs/gcn_head*bpe_vocabs*"))

In [20]:
gcn_dict = {}
reg = re.compile(r".*gcn_head_vocab_(.*)_bpe_vocabs_(.*)_neg_sample_1_vocab_size_(.*)_grid_search")
for f_name in f_names:
    with open(f_name, "r") as f:
        res_d = parse_file(f)
    m = reg.match(f_name)
    
    word_vocab = m.group(1)
    bpe_vocabs = m.group(2)
    word_vocab_size = m.group(3)
    d = {'head_vocab': word_vocab, "word_vocab_size": word_vocab_size, "side_vocabs": bpe_vocabs}
    for key, losses in res_d.items():
        d.update(dict([param.strip().split(':') for param in key.split(',')]))
        for epoch, loss_d in enumerate(losses):
            new_d = merge_dicts({'epoch': epoch, 'model': 'gcn'}, loss_d)
            gcn_pd_list.append(merge_dicts(d, new_d))

In [21]:
accs = sorted([float(rec['Accuracy'].strip()) for rec in gcn_pd_list], reverse=True)
len(accs), accs[:10]

(16320,
 [0.9614,
  0.961,
  0.9601,
  0.9596,
  0.9595,
  0.9592,
  0.9592,
  0.9587,
  0.9585,
  0.9584])

## Analyze Dataframe

In [22]:
df = pd.DataFrame.from_dict(bilstm_pd_list + gcn_pd_list)

In [23]:
df.head()

,AGGR,Accuracy,DP,Epoch,LR,NUM_LAYERS,Train Loss,Valid Loss,WD,epoch,head_vocab,model,side_vocabs,word_vocab_size
0,NA,tensor(0.5203),0.3,NaN,0.01,2,0.678797,0.710672,0.0005,0,500,bilstm,NA,-1
1,NA,tensor(0.4940),0.3,NaN,0.01,2,0.667404,0.684273,0.0005,1,500,bilstm,NA,-1
2,NA,tensor(0.5798),0.3,NaN,0.01,2,0.653367,0.651956,0.0005,2,500,bilstm,NA,-1
3,NA,tensor(0.5810),0.3,NaN,0.01,2,0.649096,0.649752,0.0005,3,500,bilstm,NA,-1
4,NA,tensor(0.5790),0.3,NaN,0.01,2,0.654959,0.676428,0.0005,4,500,bilstm,NA,-1


In [24]:
df ['Accuracy'] = df['Accuracy'].apply(lambda x: x.numpy() if isinstance(x, torch.Tensor) else float(x))

In [25]:
df.groupby(['model']).max()['Accuracy']

model
bilstm    0.961529
gcn       0.961400
Name: Accuracy, dtype: float64

In [26]:
df_gcn = df[df["model"] == "gcn"]
df_bilstm = df[df["model"] == "bilstm"]

In [31]:
df.groupby(['head_vocab']).max()[['Accuracy', 'side_vocabs']]

,Accuracy,side_vocabs
head_vocab,,
10000,0.957741,NA
500,0.961529,NA
50000,0.837955,NA
words-lower,0.961400,NA
words-mix,0.960100,NA


In [29]:
df['side_vocabs'].unique()

array(['NA', '500', '10000', '50000', '50000-10000-500', '50000-10000',
       '500-10000', '500-50000'], dtype=object)

In [30]:
df.groupby(['side_vocabs']).max()['Accuracy']

side_vocabs
10000              0.956200
500                0.952900
500-10000          0.954500
500-50000          0.961400
50000              0.957700
50000-10000        0.954700
50000-10000-500    0.953400
NA                 0.961529
Name: Accuracy, dtype: float64

In [33]:
df[['Accuracy', 'DP', 'LR', 'Train Loss', 'Valid Loss', 'WD']].astype('float32').corr()

,Accuracy,DP,LR,Train Loss,Valid Loss,WD
Accuracy,1.000000,-0.056601,-0.331849,-0.165755,-0.173204,-0.207216
DP,-0.056601,1.000000,0.174094,0.022371,0.027693,0.115207
LR,-0.331849,0.174094,1.000000,0.197108,0.216322,0.193904
Train Loss,-0.165755,0.022371,0.197108,1.000000,0.941466,0.119574
Valid Loss,-0.173204,0.027693,0.216322,0.941466,1.000000,0.128708
WD,-0.207216,0.115207,0.193904,0.119574,0.128708,1.000000


## Simplified

In [110]:
f_names = glob(join(data_dir, "grid_search/model_gcn*"))

In [113]:
records = []
for f_name in f_names:
    with open(f_name, "rb") as f:
        records.extend(pickle.load(f))

In [115]:
df = pd.DataFrame.from_dict(records)

In [116]:
df.head()

,Accuracy,DP,Epoch,LR,NUM_LAYERS,Train Loss,Valid Loss,WD,main_vocab,model,side_vocabs,word_vocab_size
0,0.804297,0.3,0,0.001,2,1.187878,0.429437,5e-05,words-lower,gcn,"10000,500",25000
1,0.853190,0.3,1,0.001,2,0.392437,0.343223,5e-05,words-lower,gcn,"10000,500",25000
2,0.878646,0.3,2,0.001,2,0.334639,0.300368,5e-05,words-lower,gcn,"10000,500",25000
3,0.890365,0.3,3,0.001,2,0.299826,0.269641,5e-05,words-lower,gcn,"10000,500",25000
4,0.897786,0.3,4,0.001,2,0.275134,0.255407,5e-05,words-lower,gcn,"10000,500",25000


In [117]:
df.groupby('main_vocab').max()['Accuracy']

main_vocab
10000          0.944987
500            0.942383
50000          0.943099
words-lower    0.944596
words-mix      0.942578
Name: Accuracy, dtype: float64

In [118]:
df.groupby('side_vocabs').max()[['Accuracy', 'main_vocab']]

,Accuracy,main_vocab
side_vocabs,,
10000,0.938867,words-mix
"10000,500",0.942708,words-mix
"10000,50000",0.936589,words-mix
500,0.938281,words-mix
"500,10000",0.944596,words-mix
"500,10000,50000",0.933919,words-mix
"500,50000",0.941797,words-mix
50000,0.938802,words-mix
"50000,10000",0.937305,words-mix


In [119]:
def create_num_side_vocabs(row):
    if row['side_vocabs'] is None:
        return  0
    else:
        return len(row['side_vocabs'].split(','))

In [120]:
df['num_side_vocabs'] = df.apply (lambda row: create_num_side_vocabs(row), axis=1)

In [121]:
df.head()

,Accuracy,DP,Epoch,LR,NUM_LAYERS,Train Loss,Valid Loss,WD,main_vocab,model,side_vocabs,word_vocab_size,num_side_vocabs
0,0.804297,0.3,0,0.001,2,1.187878,0.429437,5e-05,words-lower,gcn,"10000,500",25000,2
1,0.853190,0.3,1,0.001,2,0.392437,0.343223,5e-05,words-lower,gcn,"10000,500",25000,2
2,0.878646,0.3,2,0.001,2,0.334639,0.300368,5e-05,words-lower,gcn,"10000,500",25000,2
3,0.890365,0.3,3,0.001,2,0.299826,0.269641,5e-05,words-lower,gcn,"10000,500",25000,2
4,0.897786,0.3,4,0.001,2,0.275134,0.255407,5e-05,words-lower,gcn,"10000,500",25000,2


In [122]:
df['side_vocabs'].fillna(0, inplace=True)

In [123]:
df.groupby('num_side_vocabs').max()[['Accuracy', 'main_vocab']]

,Accuracy,main_vocab
num_side_vocabs,,
0,0.944987,words-mix
1,0.938867,words-mix
2,0.944596,words-mix
3,0.933919,words-mix


In [124]:
df.groupby(['word_vocab_size', 'side_vocabs']).max()[['Accuracy']]

Accuracy
word_vocab_size side_vocabs              
-1              0                0.944987
 5000           0                0.931641
                10000            0.937500
                10000,500        0.942708
                10000,50000      0.934570
                500              0.938281
                500,10000        0.940365
                500,10000,50000  0.933464
                500,50000        0.941797
                50000            0.937630
                50000,10000      0.935938
                50000,500        0.936003
 10000          0                0.934310
                10000            0.938867
                10000,500        0.938672
                10000,50000      0.936328
                500              0.937695
                500,10000        0.941211
                500,10000,50000  0.933464
                500,50000        0.939193
                50000            0.938542
                50000,10000      0.937305
                50000,500        0.942578
 25000          0                0.933268
                10000            0.938346
                10000,500        0.939583
                10000,50000      0.936589
                500              0.935417
                500,10000        0.944596
                500,10000,50000  0.933919
                500,50000        0.938346
                50000            0.938802
                50000,10000      0.936328
                50000,500        0.939714

## Ranking Data

In [5]:
f_names = glob(join(data_dir, "grid_search/ranking_bilstm*"))

In [29]:
all_records = []
for f_name in f_names:
    neg_sample = f_name.split('_')[-1][0]
    with open(f_name, "rb") as f:
        records = pickle.load(f)
        for rec in records:
            rec['neg_sample'] = neg_sample
        all_records.extend(records)

In [30]:
df = pd.DataFrame.from_dict(all_records)

In [31]:
df.head()

,DP,LR,NUM_LAYERS,WD,main_vocab,model,side_vocabs,word_vocab_size,1,2,5,10,100,mrr,Valid Loss,Train Loss,Epoch,neg_sample
0,0.3,0.001,2,5e-05,50000,bilstm,None,-1,0.0017145871801635452,0.004484304932735426,0.14481667106304405,0.29873384331311,0.7325243998944869,0.078539,0.210256,0.280684,0,2
1,0.3,0.001,2,5e-05,50000,bilstm,None,-1,0.1587971511474545,0.27064099182273804,0.42284357689264046,0.5270377209179636,0.839752044315484,0.285217,0.138137,0.144355,1,2
2,0.3,0.001,2,5e-05,50000,bilstm,None,-1,0.01701398048008441,0.2788182537589027,0.4697968873648114,0.5883671854391981,0.87087839620153,0.230761,0.193008,0.119452,2,2
3,0.3,0.001,2,5e-05,50000,bilstm,None,-1,0.32761804273278816,0.42007385914006856,0.5493273542600897,0.6382221049854919,0.8790556581376945,0.432483,0.095479,0.104822,3,2
4,0.3,0.001,2,5e-05,50000,bilstm,None,-1,0.35017145871801636,0.44447375362701136,0.5704299657082564,0.6642046953310472,0.8944869427591664,0.455097,0.120971,0.094683,4,2


In [33]:
df.groupby('main_vocab').max()[['mrr', 1, 10, 100]]

,mrr,1,10,100
main_vocab,,,,
10000,0.666507,0.581245,0.823793,0.957663
500,0.645487,0.557900,0.808494,0.955421
50000,0.649333,0.559483,0.815220,0.951860
words-lower,0.566429,0.476655,0.739119,0.912160
words-mix,0.529598,0.439066,0.705487,0.902005


In [34]:
df.groupby('neg_sample').max()[['mrr', 1, 10, 100]]

,mrr,1,10,100
neg_sample,,,,
1,0.645487,0.557900,0.806647,0.949486
2,0.666507,0.581245,0.823793,0.957663
